<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/monteCarlo_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

def monte_carlo_call_price(S, K, r, sigma, T, num_simulations):
    """
    Monte Carlo simulation to calculate the European call option price.

    Args:
        S (torch.Tensor): Initial stock price.
        K (torch.Tensor): Strike price.
        r (torch.Tensor): Risk-free interest rate.
        sigma (torch.Tensor): Volatility.
        T (torch.Tensor): Time to maturity.
        num_simulations (int): Number of Monte Carlo simulations.

    Returns:
        torch.Tensor: Call option price.
    """
    # Generate standard normal random numbers
    Z = torch.randn(num_simulations, dtype=torch.float64)

    # Compute stock price at maturity ST
    ST = S * torch.exp((r - 0.5 * sigma**2) * T + sigma * torch.sqrt(T) * Z)

    # Compute payoff
    payoff = torch.nn.functional.softplus(ST - K)

    # Discounted expected payoff
    call_price = torch.mean(payoff) * torch.exp(-r * T)

    return call_price

def main():
    # Enable anomaly detection for debugging gradients
    torch.autograd.set_detect_anomaly(True)

    num_simulations = 1_000_000  # Large number for accuracy

    # Define inputs with gradient tracking where needed
    S = torch.tensor(100.0, dtype=torch.float64, requires_grad=True)  # Initial stock price
    K = torch.tensor(100.0, dtype=torch.float64)                      # Strike price
    r = torch.tensor(0.05, dtype=torch.float64, requires_grad=True)   # Risk-free interest rate
    sigma = torch.tensor(0.2, dtype=torch.float64, requires_grad=True) # Volatility
    T = torch.tensor(1.0, dtype=torch.float64, requires_grad=True)    # Time to maturity

    # Compute call option price
    price = monte_carlo_call_price(S, K, r, sigma, T, num_simulations)

    # Compute first-order gradients (Delta, Rho, Vega, Theta)
    grads = torch.autograd.grad(price, [S, r, sigma, T], retain_graph=True, create_graph=True)

    delta, rho, vega, theta = grads

    # Display results
    print(f"Call Option Price: {price.item():.5f}")
    print(f"Delta (dPrice/dS): {delta.item():.5f}")
    print(f"Rho (dPrice/dr): {rho.item():.5f}")
    print(f"Vega (dPrice/dSigma): {vega.item():.5f}")
    print(f"Theta (dPrice/dT): {theta.item():.5f}")

    # Compute Gamma (second derivative with respect to S)
    gamma = torch.autograd.grad(delta, S, retain_graph=False, create_graph=False)[0]

    print(f"Gamma (d2Price/dS2): {gamma.item():.5f}")

if __name__ == "__main__":
    main()

In [ ]:
import torch

def monte_carlo_call_price(S, K, r, sigma, T, num_simulations, num_timesteps):
    """
    Monte Carlo simulation with time-stepping to calculate the European call option price.

    Args:
        S (torch.Tensor): Initial stock price.
        K (torch.Tensor): Strike price.
        r (torch.Tensor): Risk-free interest rate.
        sigma (torch.Tensor): Volatility.
        T (torch.Tensor): Time to maturity.
        num_simulations (int): Number of Monte Carlo simulations.
        num_timesteps (int): Number of time steps for the simulation.

    Returns:
        torch.Tensor: Call option price.
    """
    dt = T / num_timesteps  # Time step size

    # Initialize stock prices for all simulations
    ST = S * torch.ones(num_simulations, dtype=torch.float64)

    # Simulate stock price paths using time-stepping
    for _ in range(num_timesteps):
        Z = torch.randn(num_simulations, dtype=torch.float64)
        ST *= torch.exp((r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Z)

    # Compute payoff at maturity
    payoff = torch.nn.functional.softplus(ST - K)

    # Discounted expected payoff
    call_price = torch.mean(payoff) * torch.exp(-r * T)

    return call_price

def main():
    # Enable anomaly detection for debugging gradients
    torch.autograd.set_detect_anomaly(True)

    num_simulations = 50_000  # Number of Monte Carlo paths
    num_timesteps = 100          # Number of time steps

    # Define inputs with gradient tracking where needed
    S = torch.tensor(100.0, dtype=torch.float64, requires_grad=True)  # Initial stock price
    K = torch.tensor(100.0, dtype=torch.float64)                      # Strike price
    r = torch.tensor(0.05, dtype=torch.float64, requires_grad=True)   # Risk-free interest rate
    sigma = torch.tensor(0.2, dtype=torch.float64, requires_grad=True) # Volatility
    T = torch.tensor(1.0, dtype=torch.float64, requires_grad=True)    # Time to maturity

    # Compute call option price with time-stepping
    price = monte_carlo_call_price(S, K, r, sigma, T, num_simulations, num_timesteps)

    # Compute first-order gradients (Delta, Rho, Vega, Theta)
    grads = torch.autograd.grad(price, [S, r, sigma, T], retain_graph=True, create_graph=True)
    delta, rho, vega, theta = grads

    # Display results
    print(f"Call Option Price: {price.item():.5f}")
    print(f"Delta (dPrice/dS): {delta.item():.5f}")
    print(f"Rho (dPrice/dr): {rho.item():.5f}")
    print(f"Vega (dPrice/dSigma): {vega.item():.5f}")
    print(f"Theta (dPrice/dT): {theta.item():.5f}")

    # Second-order Greeks
    gamma = torch.autograd.grad(delta, S, retain_graph=True, create_graph=True)[0]
    charm = torch.autograd.grad(delta, T, retain_graph=True, create_graph=True)[0]
    vanna = torch.autograd.grad(delta, sigma, retain_graph=True, create_graph=True)[0]
    vomma = torch.autograd.grad(vega, sigma, retain_graph=True, create_graph=True)[0]
    speed = torch.autograd.grad(gamma, S, retain_graph=True, create_graph=True)[0]
    zomma = torch.autograd.grad(gamma, sigma, retain_graph=True, create_graph=True)[0]
    color = torch.autograd.grad(gamma, T, retain_graph=True, create_graph=True)[0]
    dvega_dtime = torch.autograd.grad(vega, T, retain_graph=True, create_graph=True)[0]

    # Elasticity (Lambda)
    lambda_ = (delta * S) / price

    print(f"Gamma (d2Price/dS2): {gamma.item():.5f}")

    print(f"Lambda (Elasticity): {lambda_.item():.5f}")
    print(f"Charm (dDelta/dT): {charm.item():.5f}")
    print(f"Vanna (dDelta/dSigma): {vanna.item():.5f}")
    print(f"Vomma (dVega/dSigma): {vomma.item():.5f}")
    print(f"Speed (dGamma/dS): {speed.item():.5f}")
    print(f"Zomma (dGamma/dSigma): {zomma.item():.5f}")
    print(f"Color (dGamma/dT): {color.item():.5f}")
    print(f"DvegaDtime (dVega/dT): {dvega_dtime.item():.5f}")

if __name__ == "__main__":
    main()